# What are those cells doing?

### This lesson will download a dataset from the Allen Brain Institute's "Software Development Kit," or SDK.

These datasets contain calcium imaging data for various different cell types in the visual cortex of the mouse. It's likely that these cell types have different roles in the visual system -- your mission is to figure out what these roles are. You will choose a visual area, a cell type, and a type of visual stimulus:

<img src="ExperimentalDesign.png">  

### By the end of this lesson, you will be able to:
1. Choose a dataset for a particular cell type and save it locally.
2. Plot a stimulus-response curve for one neuron.

Additional information on this dataset, and how it was collected, can be found here: http://help.brain-map.org/display/observatory/Data+-+Visual+Coding


## 1. Importing toolboxes
First, we'll import the necessary toolboxes to run this code. The first chunk of "import" lines will bring in some standard toolboxes that we need. For example, "numpy" is a toolbox that has functions to work with large arrays (https://en.wikipedia.org/wiki/NumPy). The second chunk of import lines brings in some toolboxes that the Allen Brain Observatory has already packaged, to help users analyze its data.

In [1]:
# Standard toolboxes
import pprint
import numpy as np
from __future__ import print_function
import h5py
import os
# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.

# Allen specific toolboxes
import allensdk.brain_observatory.stimulus_info as stim_info
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
from allensdk.brain_observatory.natural_scenes import NaturalScenes
from allensdk.brain_observatory.drifting_gratings import DriftingGratings
from allensdk.brain_observatory.static_gratings import StaticGratings

# We will save the Brain Observatory Cache as a variable, "boc."
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
cwd = os.getcwd()
print('Successfully imported AllenSDK packages.')

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully imported AllenSDK packages.


## 2. Get a list of all possible trangenic mouse lines and brain areas
Next, we'll print all of the possible Cre lines and brain areas are that we can analyze. You'll need to use these exact names when you're trying to pull a specific one from the dataset.

More info on Cre lines can be found here:
http://help.brain-map.org/display/observatory/Transgenic+Mouse+Lines

In [3]:
# We'll save the list of cre lines as a variable, 'cre-lines'.
cre_lines = boc.get_all_cre_lines()
print("all cre lines: " + str(cre_lines))

# We'll save the list of possible structures as a variable, 'brain_areas'.
brain_areas = boc.get_all_targeted_structures()
print("all brain regions: " + str(brain_areas))

all cre lines: ['Cux2-CreERT2', 'Emx1-IRES-Cre', 'Fezf2-CreER', 'Nr5a1-Cre', 'Ntsr1-Cre_GN220', 'Pvalb-IRES-Cre', 'Rbp4-Cre_KL100', 'Rorb-IRES2-Cre', 'Scnn1a-Tg3-Cre', 'Slc17a7-IRES2-Cre', 'Sst-IRES-Cre', 'Tlx3-Cre_PL56', 'Vip-IRES-Cre']
all brain regions: ['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']


Next, we'll choose one of these cell lines and brain regions to examine here.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task:</b>  Choose a visual area and Cre line from the lists above to examine in the rest of the notebook
</div>

In [4]:
# Decide which area and which cre lines you'd like to use.
# You need to insert these in the format ['VISp'] or ['Rbp4-Cre'KL100']
area = ['VISp']
cre_line = ['Rbp4-Cre_KL100']
visual_stim = 'drifting_gratings'
print("Let's take a look at how " + str(cre_line) + " cells in " + str(area) + " respond to " + visual_stim)

Let's take a look at how ['Rbp4-Cre_KL100'] cells in ['VISp'] respond to drifting_gratings


## 3. Get the response of cell types to different types of stimuli.
Now that we've chosen a cell type and brain area, we need to download a dataset that contains data for that cell type, in that brain area.

1. save df/f traces for cells in given area
2. get cells and save. 
3. skip if data already saved. 

### NOTE: _this will take about 10 minutes per dataset (if it doesn't already exist on your computer)._
<p>
<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task:</b>  Get the list of all the experiment containers for that area and Cre line combination.
</div>

In [5]:
exps = boc.get_experiment_containers(targeted_structures=[visual_area], cre_lines=[cre_line])

NameError: name 'visual_area' is not defined

In [34]:
interlength = 7 #in frames, 0.25 secs
stim_length = 7 #frames, 0.25 secs
sweep_length = stim_length + interlength

# First, we'll loop through areas (optional, only if you entered 1+ visual area)

for aa in range(len(area)):
    
    this_area = area[aa]
    
# Then, we'll look through different cre lines (optional, only if you entered 1+ cre line)

    for cc in range(len(cre_line)):
        
        this_cre_line = cre_line[cc]
        
        # We'll set up some empty variables that we'll fill later.
        X_ALL = []
        XT_ALL = []
        XT_ALL_avg = []
        Y_ALL = []
        cells_all = []
        total_num_cells = 0

        exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.DRIFTING_GRATINGS],
                                         cre_lines=[this_cre_line],)
        
        # We could loop through multiple experiments, but for simplicity here, we'll just look at the first one.
        # num_expts = len(exps)
        num_expts = 1;
        
        for ee in range(num_expts):
            
            print(ee)
            
            exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.DRIFTING_GRATINGS],
                                         cre_lines=[this_cre_line])[ee] #for this experiment
            expData = boc.get_ophys_experiment_data(exps['id']) # get experiment data - this is the time consuming step
            time,dffTraces = expData.get_dff_traces() #get traces . dffTraces is cells by time
    
            stim_table = expData.get_stimulus_table('drifting_gratings')

            cell_ids = expData.get_cell_specimen_ids()
            numCells = len(cell_ids)
            total_num_cells = numCells + total_num_cells

            count = 0

            #get 0 ori trials
            trial_mask = stim_table.orientation == 0
            this_stim_table = stim_table[trial_mask]
            trials_per_ori = len(this_stim_table)
            XT_zero = np.empty((trials_per_ori,numCells,sweep_length))
            XT_zero_avg = np.empty((trials_per_ori,numCells))
            stimID_zero = np.empty((trials_per_ori))
        
            for t in np.arange(0,trials_per_ori):
                stimID_zero[t] = 0
                for cell in np.arange(0,numCells):
                    stim_on = this_stim_table.start.iloc[t-1] 
                    # start_sweep = stim_on - interlength #start 0.25 secs (7 frames) before stimulus onset
                    end_sweep = stim_on + stim_length + interlength #end 0.25 secs (14 frames) after stimulus onset
                    trange = np.arange(stim_on,end_sweep)
                    XT_zero[t,cell,:] = dffTraces[cell,trange]
                    mean_this_cell = np.mean(XT_zero[t,cell,:])
                    XT_zero_avg[t,cell] = mean_this_cell
                    
            #get 90 ori trials
            trial_mask = stim_table.orientation == 90
            this_stim_table = stim_table[trial_mask]
            trials_per_ori = len(this_stim_table)
            XT_ninety = np.empty((trials_per_ori,numCells,sweep_length))
            XT_ninety_avg = np.empty((trials_per_ori,numCells))
            stimID_ninety = np.empty((trials_per_ori))
       
            for t in np.arange(0,trials_per_ori):
                stimID_ninety[t] = 90
                for cell in np.arange(0,numCells):
                    stim_on = this_stim_table.start.iloc[t-1] 
                    # start_sweep = stim_on - interlength #start 0.25 secs (7 frames) before stimulus onset
                    end_sweep = stim_on + stim_length + interlength #end 0.25 secs (14 frames) after stimulus onset
                    # end_sweep = stim_on + stim_length
                    # trange = np.arange(stim_on+6,stim_on+10) # range for 200-300 ms post stimulus period
                    trange = np.arange(stim_on,end_sweep)
                    XT_ninety[t,cell,:] = dffTraces[cell,trange]
                    mean_this_cell = np.mean(XT_ninety[t,cell,:])
                    XT_ninety_avg[t,cell] = mean_this_cell
        
            # concatenate 0 and 90 experiments
            XT_ALL = np.vstack((XT_zero,XT_ninety))
            XT_ALL_avg = np.vstack((XT_zero_avg,XT_ninety_avg))
            stimID_ALL = np.hstack((stimID_zero,stimID_ninety))
        
            print(this_area + ',  ' + this_cre_line + ': ' + str(numCells) + ' cells')
            filename = this_area +'_'+ this_cre_line + str(exps['id']) + '_staticOri_data.h5'

            #save arrays X_ALL & XT_ALL
            with h5py.File(filename,'w') as hf:
                hf.create_dataset('X_matrix_time', data=XT_ALL,chunks=True)
                hf.create_dataset('XT_stimPeriodavg', data=XT_ALL_avg,chunks=True)
                hf.create_dataset('stim_vector', data=stimID_ALL,chunks=True)
            

2018-09-20 13:57:14,010 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/540020118


0


NameError: name 'drifting_gratings' is not defined

str

## get natural scenes delta F/F
1. save df/f traces for cells in given area
2. get cells and save. 
3. skip if data already saved. 

_this will take a bit of time_

In [35]:
interlength = 7 #in frames, 0.25 secs
stim_length = 7 #frames, 0.25 secs
sweep_length = 2*interlength+stim_length #in frames, 1secs

for aa in range(len(areas)):
    this_area = areas[aa]
    
    for cc in range(len(cre_lines)):
        this_cre_line = cre_lines[cc]
        X_ALL = []
        XT_ALL = []
        Y_ALL = []
        cells_all = []
        total_num_cells = 0
        
        exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.NATURAL_SCENES],
                                cre_lines=[this_cre_line],)
        num_exps = len(exps)

        for ee in range(num_exps):
            
            exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.NATURAL_SCENES],
                                             cre_lines=[this_cre_line])[ee]
            expData = boc.get_ophys_experiment_data(exps['id'])
            time,dffTraces = expData.get_dff_traces() #get traces . dffTraces is cells by time

            #118 scenes, each scene presented 50 times. there is a scene labeled -1 in stimulus table, must be blank
            stim_table = expData.get_stimulus_table('natural_scenes')
            scenes = expData.get_stimulus_template('natural_scenes')
            sceneIDs = np.unique(stim_table.frame)
            cell_ids = expData.get_cell_specimen_ids()
            numCells = len(cell_ids)
            total_num_cells = numCells + total_num_cells
            nTrials = len(stim_table)
            y = np.empty((nTrials))
            y[:] = np.nan
            trials_per_scene = nTrials/len(sceneIDs)
            XT_exp = np.empty((nTrials,numCells,sweep_length))

            for scene in sceneIDs:
                trial_mask = (stim_table.frame == scene)
                this_stim_table = stim_table[trial_mask]
                for t in np.arange(1,trials_per_scene+1):
                    trial = t + (scene + 1) * trials_per_scene - 1
                    y[trial] = scene + 1
                    for cell in np.arange(0,numCells):
                        stim_on = this_stim_table.start.iloc[t-1] 
                        start_sweep = stim_on - interlength #start one second (28 frames) before stimulus onset
                        end_sweep = stim_on + stim_length + interlength   #end 
                        trange = np.arange(start_sweep ,end_sweep)
                        XT_exp[trial,cell,:] = dffTraces[cell,trange]

            #concatenate experiments
            if ee == 0:
                XT_ALL = XT_exp
                Y_ALL = y
                cells_all = cell_ids
            if ee > 0:
                XT_ALL = np.hstack((XT_ALL,XT_exp))
                Y_ALL = np.vstack((Y_ALL,y))
                cells_all = np.hstack((cells_all,cell_ids))

        print(this_area + ',  ' + this_cre_line + ': ' + str(numCells) + ' cells')
        filename = this_area +'_'+ this_cre_line + '_' + str(exps['id']) + '_naturalScenes.h5'
    
        #save arrays X_ALL & XT_ALL -make this into function
        with h5py.File(filename,'w') as hf:
            hf.create_dataset('X_matrix_time', data=XT_exp,chunks=True)
            hf.create_dataset('Y_matrix', data=y,chunks=True)
            hf.create_dataset('cell_IDs', data=cell_ids,chunks=True)

VISp,  Rbp4-Cre_KL100: 76 cells
VISp,  Cux2-CreERT2: 145 cells
VISl,  Rbp4-Cre_KL100: 45 cells
VISl,  Cux2-CreERT2: 99 cells


array([113806, 113807, 113808, 113809, 113810, 113811, 113812])